# Full Text Screening
**Author:** Jack Galbraith-Edge

In [3]:
# Import libraries
from msc_code.scripts.notebook_setup import *

In [5]:
# Import results that were included from Title and Abstract Review.
import_path = os.path.join(PROC_DATA_DIR, "title_abstract_review", "title_abstract_review_include_final.csv") # Define import path.
title_abstract_included = pd.read_csv(import_path) # Import CSV to Pandas Dataframe.

# Take 'id', 'Publication Year', 'Authors', 'Title', 'Publication Title', 'Database' colums to create a new dataframe.
full_text_review_start_df = title_abstract_included[['id', 'Publication Year', 'Authors', 'Title', 'Publication Title', 'Database']]

# Add 'Exclude', 'Reason ID' and 'Comments' columns.
full_text_review_start_df['Exclude'] = None
full_text_review_start_df['Reason ID'] = None
full_text_review_start_df['Comments'] = None

# View first five rows of dataframe
full_text_review_start_df.head()

# Save the dataframe to a CSV file for full text screening.
full_text_review_start_df.to_csv("/".join([PROC_DATA_DIR, "full_text_screen", "full_text_screen_start.csv"]), index=False)

/var/folders/2z/pkyt34z93md62kmcdphb_kf00000gn/T/ipykernel_7302/2674317898.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_text_review_start_df['Exclude'] = None
/var/folders/2z/pkyt34z93md62kmcdphb_kf00000gn/T/ipykernel_7302/2674317898.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_text_review_start_df['Reason ID'] = None
/var/folders/2z/pkyt34z93md62kmcdphb_kf00000gn/T/ipykernel_7302/2674317898.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Dat

In [ ]:
# Reimport Screened Results as Pandas dataframe.
